In [ ]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="enreg/config/", job_name="test_app"):
    cfg = compose(config_name="benchmarking")

In [ ]:
from enreg.tools.metrics import decay_mode_evaluator as dme
from enreg.tools.metrics import regression_evaluator as re
from enreg.tools.metrics import tagger_evaluator as te
from enreg.tools import general as g
import os
import awkward as ak

In [ ]:
cfg.comparison_samples = ['zh_test', 'z_test', 'qq_test']
algorithms = ["DeepSet"]
tasks = ["binary_classification", "jet_regression", "dm_multiclass"]
signal_samples = ['zh_test', 'z_test']

In [ ]:
data = {sample: g.load_all_data(os.path.join(cfg.base_ntuple_path, sample + ".parquet")) for sample in cfg.comparison_samples}

In [ ]:
task = "binary_classification"
evaluators = []
for algorithm in algorithms:
    base_path = os.path.join("/home/laurits/ml-tau-en-reg/training-outputs/Trainings/v1", task, algorithm)
    bkg_data = g.load_all_data(os.path.join(base_path, "qq_test.parquet"))
    for signal_sample in signal_samples:
        sig_info_data = data[signal_sample]
        bkg_info_data = data['qq_test']
        sig_data = g.load_all_data(os.path.join(base_path, signal_sample + ".parquet"))
        
        evaluator = te.TaggerEvaluator(
            signal_predictions=sig_data.binary_classification.pred,
            signal_truth=sig_data.binary_classification.target,
            signal_gen_tau_p4=sig_info_data.gen_jet_tau_p4s,
            signal_reco_jet_p4=sig_info_data.reco_jet_p4s,
            bkg_predictions=bkg_data.binary_classification.pred,
            bkg_truth=bkg_data.binary_classification.target,
            bkg_gen_jet_p4=bkg_info_data.gen_jet_p4s,
            bkg_reco_jet_p4=bkg_info_data.reco_jet_p4s,
            cfg=cfg.metrics.classifier,
            sample=signal_sample,
            algorithm=algorithm
        )
        evaluators.append(evaluator)

tme = te.TaggerMultiEvaluator("output_plots_cls", cfg.metrics.classifier)
tme.combine_results(evaluators)
tme.save_results()

In [ ]:
task = "dm_multiclass"
for algorithm in algorithms:
    base_path = os.path.join("/home/laurits/ml-tau-en-reg/training-outputs/Trainings/v1", task, algorithm)
    for signal_sample in signal_samples:
        sig_info_data = data[signal_sample]
        sig_data = g.load_all_data(os.path.join(base_path, signal_sample + ".parquet"))

        output_dir = "output_plots_dm"
        evaluator = dme.DecayModeEvaluator(sig_data.dm_multiclass.pred, sig_data.dm_multiclass.target, output_dir, signal_sample, algorithm)
        evaluator.save_performance()



In [ ]:
task = "jet_regression"
evaluators = []
for algorithm in algorithms:
    base_path = os.path.join("/home/laurits/ml-tau-en-reg/training-outputs/Trainings/v1", task, algorithm)
    for signal_sample in signal_samples:
        sig_info_data = data[signal_sample]
        sig_data = g.load_all_data(os.path.join(base_path, signal_sample + ".parquet"))

        evaluator = re.RegressionEvaluator(sig_data.jet_regression.pred, sig_data.jet_regression.target, cfg.metrics.regression, signal_sample.split("_")[0], algorithm)
        evaluators.append(evaluator)
output_dir = "output_plots_regression"
rme = re.RegressionMultiEvaluator(output_dir, cfg.metrics.regression, signal_sample.split("_")[0])
rme.combine_results(evaluators)
rme.save()